### Import packages

In [1]:
# !pip install rasterio

In [2]:
# !pip install tqdm==4.33.0

In [3]:
# !pip show tqdm

In [4]:
# !pip install geopandas

In [5]:
import os
import requests, zipfile, io
import numpy as np # linear algebra
import pandas as pd

import rasterio
from rasterio.plot import show
from rasterio.plot import show_hist
import matplotlib.pyplot as plt        
%matplotlib inline
from matplotlib.pyplot import imshow
from scipy import stats
from rasterio.merge import merge
import geopandas as gpd
from rasterio.mask import mask
from rasterio.plot import adjust_band
from rasterio.plot import reshape_as_raster, reshape_as_image
from shapely.geometry import mapping
import sys
from tqdm import tqdm
tqdm.pandas()
import gc
import psutil

from queue import PriorityQueue

/anaconda/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


## Import shapefiles

In [6]:
os.listdir('data/test')

['test.dbf',
 '.ipynb_checkpoints',
 'test.shp',
 'test.cpg',
 'test.prj',
 'test.shx',
 'test.qpj']

In [7]:
train = gpd.read_file('data/train/train.shp')
train.dropna(subset=['geometry'], inplace = True)
train.reset_index(drop=True, inplace= True)
test = gpd.read_file('data/test/test.shp')
sample_sub = pd.read_csv('data/sample_submission.csv')

train_fe = train.copy()
test_fe = test.copy()

In [8]:
train_fe.head()

,Field_Id,Area,Subregion,Crop_Id_Ne,geometry
0,1,0.915905,3,8,POLYGON ((2467881.175041331 -3352032.059296422...
1,2,2.064410,3,6,"POLYGON ((2468428.90623293 -3351993.17305059, ..."
2,3,1.080300,3,8,POLYGON ((2468333.745293362 -3352058.485056543...
3,4,1.316190,3,8,POLYGON ((2468607.857733211 -3352012.337531495...
4,7,5.529220,3,8,"POLYGON ((2468646.08520868 -3351706.201920062,..."


### Folder names

In [9]:
date1 = '2017-01-01'
date2 = "2017-01-31"
date3 = "2017-02-10"
date4 = "2017-03-12"
date5 = "2017-03-22"
date6 = "2017-05-31"
date7 = "2017-06-20"
date8 = "2017-07-10"
date9 = "2017-07-15"
date10 = "2017-08-04"
date11 = "2017-08-19"

In [10]:
date1_pf1 = 'S2A_MSIL1C_20170101T082332_N0204_R121_T34JEP_20170101T084543.SAFE'
date1_pf2 = 'S2A_MSIL1C_20170101T082332_N0204_R121_T34JFP_20170101T084543.SAFE'

date2_pf1 = 'S2A_MSIL1C_20170131T082151_N0204_R121_T34JEP_20170131T084118.SAFE'
date2_pf2 = 'S2A_MSIL1C_20170131T082151_N0204_R121_T34JFP_20170131T084118.SAFE'

date3_pf1 = 'S2A_MSIL1C_20170210T082051_N0204_R121_T34JEP_20170210T083752.SAFE'
date3_pf2 = 'S2A_MSIL1C_20170210T082051_N0204_R121_T34JFP_20170210T083752.SAFE'

date4_pf1 = 'S2A_MSIL1C_20170312T082001_N0204_R121_T34JEP_20170312T084235.SAFE '
date4_pf2 = 'S2A_MSIL1C_20170312T082001_N0204_R121_T34JFP_20170312T084235.SAFE'

date5_pf1 = 'S2A_MSIL1C_20170322T081611_N0204_R121_T34JEP_20170322T084728.SAFE'
date5_pf2 = 'S2A_MSIL1C_20170322T081611_N0204_R121_T34JFP_20170322T084728.SAFE'

date6_pf1 = 'S2A_MSIL1C_20170531T082011_N0205_R121_T34JEP_20170531T084246.SAFE'
date6_pf2 = 'S2A_MSIL1C_20170531T082011_N0205_R121_T34JFP_20170531T084246.SAFE'

date7_pf1 = 'S2A_MSIL1C_20170620T082011_N0205_R121_T34JEP_20170620T084200.SAFE'
date7_pf2 = 'S2A_MSIL1C_20170620T082011_N0205_R121_T34JFP_20170620T084200.SAFE'

date8_pf1 = 'S2A_MSIL1C_20170710T082011_N0205_R121_T34JEP_20170710T084244.SAFE'
date8_pf2 = 'S2A_MSIL1C_20170710T082011_N0205_R121_T34JFP_20170710T084244.SAFE'

date9_pf1 = 'S2B_MSIL1C_20170715T081609_N0205_R121_T34JEP_20170715T084650.SAFE'
date9_pf2 = 'S2B_MSIL1C_20170715T081609_N0205_R121_T34JFP_20170715T084650.SAFE'

date10_pf1 = 'S2B_MSIL1C_20170804T081559_N0205_R121_T34JEP_20170804T084631.SAFE'
date10_pf2 = 'S2B_MSIL1C_20170804T081559_N0205_R121_T34JFP_20170804T084631.SAFE'

date11_pf1 = 'S2A_MSIL1C_20170819T082011_N0205_R121_T34JEP_20170819T084427.SAFE'
date11_pf2 = 'S2A_MSIL1C_20170819T082011_N0205_R121_T34JFP_20170819T084427.SAFE'

In [11]:
for dirname, _, filenames in os.walk('data/'+date1_pf1):
    for filename in filenames:
        print(os.path.join(dirname, filename))

data/S2A_MSIL1C_20170101T082332_N0204_R121_T34JEP_20170101T084543.SAFE/INSPIRE.xml
data/S2A_MSIL1C_20170101T082332_N0204_R121_T34JEP_20170101T084543.SAFE/manifest.safe
data/S2A_MSIL1C_20170101T082332_N0204_R121_T34JEP_20170101T084543.SAFE/MTD_MSIL1C.xml
data/S2A_MSIL1C_20170101T082332_N0204_R121_T34JEP_20170101T084543.SAFE/rep_info/S2_User_Product_Level-1C_Metadata.xsd
data/S2A_MSIL1C_20170101T082332_N0204_R121_T34JEP_20170101T084543.SAFE/DATASTRIP/DS_MTI__20170103T030708_S20170101T084543/MTD_DS.xml
data/S2A_MSIL1C_20170101T082332_N0204_R121_T34JEP_20170101T084543.SAFE/HTML/star_bg.jpg
data/S2A_MSIL1C_20170101T082332_N0204_R121_T34JEP_20170101T084543.SAFE/HTML/UserProduct_index.xsl
data/S2A_MSIL1C_20170101T082332_N0204_R121_T34JEP_20170101T084543.SAFE/HTML/UserProduct_index.html
data/S2A_MSIL1C_20170101T082332_N0204_R121_T34JEP_20170101T084543.SAFE/HTML/banner_3.png
data/S2A_MSIL1C_20170101T082332_N0204_R121_T34JEP_20170101T084543.SAFE/HTML/banner_2.png
data/S2A_MSIL1C_20170101T082332_

## Create mosaic

In [13]:
def create_mosaic(fp_bands, sp_bands, date, directory = ''):
    """
    This function creates a mosaic from two raster images and saves it to the directory with the name Mosaic_"+ date+ ".tiff.
    The mosaic will have a number of bands equal to the ones specified in the fp_bands.
    fp_bands: list:
    list of first raster bands that we use to construct the first raster
    sp_bands: list:
    list of the second raster bands that we use to construct the second raster
    date: string:
    the date of the raster, we use it to name the final registered the mosaic
    """
    
    if(len(fp_bands) == 0 or len(sp_bands)== 0):
        print('There are no bands')
        return 0
    if(len(fp_bands) != len(sp_bands)):
        print('The length of the two bands lists are not equal')
        return 0
    
    ## set the first part of the mosaic image
    nb_bands = len(fp_bands)
    with rasterio.open(fp_bands[0]) as src0:
        fb_meta = src0.meta

    fb_meta['driver'] = 'Gtiff'
    fb_meta['count'] = nb_bands
    
    #export true color image
    with rasterio.open('first_part.tiff', 'w', **fb_meta) as dst:
        for id, layer in enumerate(fp_bands, start=1):
            with rasterio.open(layer) as src1:
                dst.write(src1.read(1), id)

    print('first part completed')
    
    ## set the second part of the mosaic image

    with rasterio.open(sp_bands[0]) as src0:
        sb_meta = src0.meta

    sb_meta['driver'] = 'Gtiff'
    sb_meta['count'] = nb_bands
    
    #export true color image
    with rasterio.open('second_part.tiff', 'w', **sb_meta) as dst:
        for id, layer in enumerate(sp_bands, start=1):
            with rasterio.open(layer) as src1:
                dst.write(src1.read(1), id)
    print('second part completed')
    
    ## Create the mosaic
    first_part = rasterio.open(r"first_part.tiff", count=nb_bands)
    second_part = rasterio.open(r"second_part.tiff", count=nb_bands)
    os.remove(r"./first_part.tiff")
    os.remove(r"./second_part.tiff")

    src_files_to_mosaic = [first_part, second_part]  
    mosaic, out_trans = merge(src_files_to_mosaic)
    
    out_meta = sb_meta.copy()
    out_meta.update({"height": mosaic.shape[1],
                    "width": mosaic.shape[2],
                    "transform": out_trans})

    out_fp = os.path.join('mosaics_allbands', r"Mosaic_"+ date+ ".tiff")
    with rasterio.open(out_fp, "w", **out_meta) as dest:
        dest.write(mosaic)
        
        
    first_part.close()
    second_part.close()
    gc.collect()
    print('mosaic is created and saved')

In [15]:
### 2017-01-01 mosiac

parent_path = 'data'

fp_bands = []
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170101T082332_N0204_R121_T34JEP_20170101T084543.SAFE/GRANULE/L1C_T34JEP_A007983_20170101T084543/IMG_DATA/T34JEP_20170101T082332_B02.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170101T082332_N0204_R121_T34JEP_20170101T084543.SAFE/GRANULE/L1C_T34JEP_A007983_20170101T084543/IMG_DATA/T34JEP_20170101T082332_B03.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170101T082332_N0204_R121_T34JEP_20170101T084543.SAFE/GRANULE/L1C_T34JEP_A007983_20170101T084543/IMG_DATA/T34JEP_20170101T082332_B04.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170101T082332_N0204_R121_T34JEP_20170101T084543.SAFE/GRANULE/L1C_T34JEP_A007983_20170101T084543/IMG_DATA/T34JEP_20170101T082332_B05.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170101T082332_N0204_R121_T34JEP_20170101T084543.SAFE/GRANULE/L1C_T34JEP_A007983_20170101T084543/IMG_DATA/T34JEP_20170101T082332_B06.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170101T082332_N0204_R121_T34JEP_20170101T084543.SAFE/GRANULE/L1C_T34JEP_A007983_20170101T084543/IMG_DATA/T34JEP_20170101T082332_B07.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170101T082332_N0204_R121_T34JEP_20170101T084543.SAFE/GRANULE/L1C_T34JEP_A007983_20170101T084543/IMG_DATA/T34JEP_20170101T082332_B08.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170101T082332_N0204_R121_T34JEP_20170101T084543.SAFE/GRANULE/L1C_T34JEP_A007983_20170101T084543/IMG_DATA/T34JEP_20170101T082332_B8A.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170101T082332_N0204_R121_T34JEP_20170101T084543.SAFE/GRANULE/L1C_T34JEP_A007983_20170101T084543/IMG_DATA/T34JEP_20170101T082332_B11.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170101T082332_N0204_R121_T34JEP_20170101T084543.SAFE/GRANULE/L1C_T34JEP_A007983_20170101T084543/IMG_DATA/T34JEP_20170101T082332_B12.jp2'))

sp_bands = []

sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170101T082332_N0204_R121_T34JFP_20170101T084543.SAFE/GRANULE/L1C_T34JFP_A007983_20170101T084543/IMG_DATA/T34JFP_20170101T082332_B02.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170101T082332_N0204_R121_T34JFP_20170101T084543.SAFE/GRANULE/L1C_T34JFP_A007983_20170101T084543/IMG_DATA/T34JFP_20170101T082332_B03.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170101T082332_N0204_R121_T34JFP_20170101T084543.SAFE/GRANULE/L1C_T34JFP_A007983_20170101T084543/IMG_DATA/T34JFP_20170101T082332_B04.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170101T082332_N0204_R121_T34JFP_20170101T084543.SAFE/GRANULE/L1C_T34JFP_A007983_20170101T084543/IMG_DATA/T34JFP_20170101T082332_B05.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170101T082332_N0204_R121_T34JFP_20170101T084543.SAFE/GRANULE/L1C_T34JFP_A007983_20170101T084543/IMG_DATA/T34JFP_20170101T082332_B06.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170101T082332_N0204_R121_T34JFP_20170101T084543.SAFE/GRANULE/L1C_T34JFP_A007983_20170101T084543/IMG_DATA/T34JFP_20170101T082332_B07.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170101T082332_N0204_R121_T34JFP_20170101T084543.SAFE/GRANULE/L1C_T34JFP_A007983_20170101T084543/IMG_DATA/T34JFP_20170101T082332_B08.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170101T082332_N0204_R121_T34JFP_20170101T084543.SAFE/GRANULE/L1C_T34JFP_A007983_20170101T084543/IMG_DATA/T34JFP_20170101T082332_B8A.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170101T082332_N0204_R121_T34JFP_20170101T084543.SAFE/GRANULE/L1C_T34JFP_A007983_20170101T084543/IMG_DATA/T34JFP_20170101T082332_B11.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170101T082332_N0204_R121_T34JFP_20170101T084543.SAFE/GRANULE/L1C_T34JFP_A007983_20170101T084543/IMG_DATA/T34JFP_20170101T082332_B12.jp2'))


date = '2017-01-01'
create_mosaic(fp_bands, sp_bands, date)

first part completed
second part completed
mosaic is created and saved


In [16]:
### 2017-01-31 mosiac
fp_bands = []
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170131T082151_N0204_R121_T34JEP_20170131T084118.SAFE/GRANULE/L1C_T34JEP_A008412_20170131T084118/IMG_DATA/T34JEP_20170131T082151_B02.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170131T082151_N0204_R121_T34JEP_20170131T084118.SAFE/GRANULE/L1C_T34JEP_A008412_20170131T084118/IMG_DATA/T34JEP_20170131T082151_B03.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170131T082151_N0204_R121_T34JEP_20170131T084118.SAFE/GRANULE/L1C_T34JEP_A008412_20170131T084118/IMG_DATA/T34JEP_20170131T082151_B04.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170131T082151_N0204_R121_T34JEP_20170131T084118.SAFE/GRANULE/L1C_T34JEP_A008412_20170131T084118/IMG_DATA/T34JEP_20170131T082151_B05.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170131T082151_N0204_R121_T34JEP_20170131T084118.SAFE/GRANULE/L1C_T34JEP_A008412_20170131T084118/IMG_DATA/T34JEP_20170131T082151_B06.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170131T082151_N0204_R121_T34JEP_20170131T084118.SAFE/GRANULE/L1C_T34JEP_A008412_20170131T084118/IMG_DATA/T34JEP_20170131T082151_B07.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170131T082151_N0204_R121_T34JEP_20170131T084118.SAFE/GRANULE/L1C_T34JEP_A008412_20170131T084118/IMG_DATA/T34JEP_20170131T082151_B08.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170131T082151_N0204_R121_T34JEP_20170131T084118.SAFE/GRANULE/L1C_T34JEP_A008412_20170131T084118/IMG_DATA/T34JEP_20170131T082151_B8A.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170131T082151_N0204_R121_T34JEP_20170131T084118.SAFE/GRANULE/L1C_T34JEP_A008412_20170131T084118/IMG_DATA/T34JEP_20170131T082151_B11.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170131T082151_N0204_R121_T34JEP_20170131T084118.SAFE/GRANULE/L1C_T34JEP_A008412_20170131T084118/IMG_DATA/T34JEP_20170131T082151_B12.jp2'))


sp_bands = []
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170131T082151_N0204_R121_T34JFP_20170131T084118.SAFE/GRANULE/L1C_T34JFP_A008412_20170131T084118/IMG_DATA/T34JFP_20170131T082151_B02.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170131T082151_N0204_R121_T34JFP_20170131T084118.SAFE/GRANULE/L1C_T34JFP_A008412_20170131T084118/IMG_DATA/T34JFP_20170131T082151_B03.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170131T082151_N0204_R121_T34JFP_20170131T084118.SAFE/GRANULE/L1C_T34JFP_A008412_20170131T084118/IMG_DATA/T34JFP_20170131T082151_B04.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170131T082151_N0204_R121_T34JFP_20170131T084118.SAFE/GRANULE/L1C_T34JFP_A008412_20170131T084118/IMG_DATA/T34JFP_20170131T082151_B05.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170131T082151_N0204_R121_T34JFP_20170131T084118.SAFE/GRANULE/L1C_T34JFP_A008412_20170131T084118/IMG_DATA/T34JFP_20170131T082151_B06.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170131T082151_N0204_R121_T34JFP_20170131T084118.SAFE/GRANULE/L1C_T34JFP_A008412_20170131T084118/IMG_DATA/T34JFP_20170131T082151_B07.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170131T082151_N0204_R121_T34JFP_20170131T084118.SAFE/GRANULE/L1C_T34JFP_A008412_20170131T084118/IMG_DATA/T34JFP_20170131T082151_B08.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170131T082151_N0204_R121_T34JFP_20170131T084118.SAFE/GRANULE/L1C_T34JFP_A008412_20170131T084118/IMG_DATA/T34JFP_20170131T082151_B8A.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170131T082151_N0204_R121_T34JFP_20170131T084118.SAFE/GRANULE/L1C_T34JFP_A008412_20170131T084118/IMG_DATA/T34JFP_20170131T082151_B11.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170131T082151_N0204_R121_T34JFP_20170131T084118.SAFE/GRANULE/L1C_T34JFP_A008412_20170131T084118/IMG_DATA/T34JFP_20170131T082151_B12.jp2'))

date = '2017-01-31'
create_mosaic(fp_bands, sp_bands, date)

first part completed
second part completed
mosaic is created and saved


In [17]:
### 2017-02-10 mosiac

fp_bands = []
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170210T082051_N0204_R121_T34JEP_20170210T083752.SAFE/GRANULE/L1C_T34JEP_A008555_20170210T083752/IMG_DATA/T34JEP_20170210T082051_B02.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170210T082051_N0204_R121_T34JEP_20170210T083752.SAFE/GRANULE/L1C_T34JEP_A008555_20170210T083752/IMG_DATA/T34JEP_20170210T082051_B03.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170210T082051_N0204_R121_T34JEP_20170210T083752.SAFE/GRANULE/L1C_T34JEP_A008555_20170210T083752/IMG_DATA/T34JEP_20170210T082051_B04.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170210T082051_N0204_R121_T34JEP_20170210T083752.SAFE/GRANULE/L1C_T34JEP_A008555_20170210T083752/IMG_DATA/T34JEP_20170210T082051_B05.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170210T082051_N0204_R121_T34JEP_20170210T083752.SAFE/GRANULE/L1C_T34JEP_A008555_20170210T083752/IMG_DATA/T34JEP_20170210T082051_B06.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170210T082051_N0204_R121_T34JEP_20170210T083752.SAFE/GRANULE/L1C_T34JEP_A008555_20170210T083752/IMG_DATA/T34JEP_20170210T082051_B07.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170210T082051_N0204_R121_T34JEP_20170210T083752.SAFE/GRANULE/L1C_T34JEP_A008555_20170210T083752/IMG_DATA/T34JEP_20170210T082051_B08.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170210T082051_N0204_R121_T34JEP_20170210T083752.SAFE/GRANULE/L1C_T34JEP_A008555_20170210T083752/IMG_DATA/T34JEP_20170210T082051_B8A.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170210T082051_N0204_R121_T34JEP_20170210T083752.SAFE/GRANULE/L1C_T34JEP_A008555_20170210T083752/IMG_DATA/T34JEP_20170210T082051_B11.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170210T082051_N0204_R121_T34JEP_20170210T083752.SAFE/GRANULE/L1C_T34JEP_A008555_20170210T083752/IMG_DATA/T34JEP_20170210T082051_B12.jp2'))


sp_bands = []
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170210T082051_N0204_R121_T34JFP_20170210T083752.SAFE/GRANULE/L1C_T34JFP_A008555_20170210T083752/IMG_DATA/T34JFP_20170210T082051_B02.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170210T082051_N0204_R121_T34JFP_20170210T083752.SAFE/GRANULE/L1C_T34JFP_A008555_20170210T083752/IMG_DATA/T34JFP_20170210T082051_B03.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170210T082051_N0204_R121_T34JFP_20170210T083752.SAFE/GRANULE/L1C_T34JFP_A008555_20170210T083752/IMG_DATA/T34JFP_20170210T082051_B04.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170210T082051_N0204_R121_T34JFP_20170210T083752.SAFE/GRANULE/L1C_T34JFP_A008555_20170210T083752/IMG_DATA/T34JFP_20170210T082051_B05.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170210T082051_N0204_R121_T34JFP_20170210T083752.SAFE/GRANULE/L1C_T34JFP_A008555_20170210T083752/IMG_DATA/T34JFP_20170210T082051_B06.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170210T082051_N0204_R121_T34JFP_20170210T083752.SAFE/GRANULE/L1C_T34JFP_A008555_20170210T083752/IMG_DATA/T34JFP_20170210T082051_B07.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170210T082051_N0204_R121_T34JFP_20170210T083752.SAFE/GRANULE/L1C_T34JFP_A008555_20170210T083752/IMG_DATA/T34JFP_20170210T082051_B08.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170210T082051_N0204_R121_T34JFP_20170210T083752.SAFE/GRANULE/L1C_T34JFP_A008555_20170210T083752/IMG_DATA/T34JFP_20170210T082051_B8A.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170210T082051_N0204_R121_T34JFP_20170210T083752.SAFE/GRANULE/L1C_T34JFP_A008555_20170210T083752/IMG_DATA/T34JFP_20170210T082051_B11.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170210T082051_N0204_R121_T34JFP_20170210T083752.SAFE/GRANULE/L1C_T34JFP_A008555_20170210T083752/IMG_DATA/T34JFP_20170210T082051_B12.jp2'))

date = '2017-02-10'
create_mosaic(fp_bands, sp_bands, date)

first part completed
second part completed
mosaic is created and saved


In [18]:
### 2017-03-12 mosiac

fp_bands = []
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170312T082001_N0204_R121_T34JEP_20170312T084235.SAFE/GRANULE/L1C_T34JEP_A008984_20170312T084235/IMG_DATA/T34JEP_20170312T082001_B02.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170312T082001_N0204_R121_T34JEP_20170312T084235.SAFE/GRANULE/L1C_T34JEP_A008984_20170312T084235/IMG_DATA/T34JEP_20170312T082001_B03.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170312T082001_N0204_R121_T34JEP_20170312T084235.SAFE/GRANULE/L1C_T34JEP_A008984_20170312T084235/IMG_DATA/T34JEP_20170312T082001_B04.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170312T082001_N0204_R121_T34JEP_20170312T084235.SAFE/GRANULE/L1C_T34JEP_A008984_20170312T084235/IMG_DATA/T34JEP_20170312T082001_B05.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170312T082001_N0204_R121_T34JEP_20170312T084235.SAFE/GRANULE/L1C_T34JEP_A008984_20170312T084235/IMG_DATA/T34JEP_20170312T082001_B06.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170312T082001_N0204_R121_T34JEP_20170312T084235.SAFE/GRANULE/L1C_T34JEP_A008984_20170312T084235/IMG_DATA/T34JEP_20170312T082001_B07.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170312T082001_N0204_R121_T34JEP_20170312T084235.SAFE/GRANULE/L1C_T34JEP_A008984_20170312T084235/IMG_DATA/T34JEP_20170312T082001_B08.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170312T082001_N0204_R121_T34JEP_20170312T084235.SAFE/GRANULE/L1C_T34JEP_A008984_20170312T084235/IMG_DATA/T34JEP_20170312T082001_B8A.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170312T082001_N0204_R121_T34JEP_20170312T084235.SAFE/GRANULE/L1C_T34JEP_A008984_20170312T084235/IMG_DATA/T34JEP_20170312T082001_B11.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170312T082001_N0204_R121_T34JEP_20170312T084235.SAFE/GRANULE/L1C_T34JEP_A008984_20170312T084235/IMG_DATA/T34JEP_20170312T082001_B12.jp2'))

sp_bands = [] 
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170312T082001_N0204_R121_T34JFP_20170312T084235.SAFE/GRANULE/L1C_T34JFP_A008984_20170312T084235/IMG_DATA/T34JFP_20170312T082001_B02.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170312T082001_N0204_R121_T34JFP_20170312T084235.SAFE/GRANULE/L1C_T34JFP_A008984_20170312T084235/IMG_DATA/T34JFP_20170312T082001_B03.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170312T082001_N0204_R121_T34JFP_20170312T084235.SAFE/GRANULE/L1C_T34JFP_A008984_20170312T084235/IMG_DATA/T34JFP_20170312T082001_B04.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170312T082001_N0204_R121_T34JFP_20170312T084235.SAFE/GRANULE/L1C_T34JFP_A008984_20170312T084235/IMG_DATA/T34JFP_20170312T082001_B05.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170312T082001_N0204_R121_T34JFP_20170312T084235.SAFE/GRANULE/L1C_T34JFP_A008984_20170312T084235/IMG_DATA/T34JFP_20170312T082001_B06.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170312T082001_N0204_R121_T34JFP_20170312T084235.SAFE/GRANULE/L1C_T34JFP_A008984_20170312T084235/IMG_DATA/T34JFP_20170312T082001_B07.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170312T082001_N0204_R121_T34JFP_20170312T084235.SAFE/GRANULE/L1C_T34JFP_A008984_20170312T084235/IMG_DATA/T34JFP_20170312T082001_B08.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170312T082001_N0204_R121_T34JFP_20170312T084235.SAFE/GRANULE/L1C_T34JFP_A008984_20170312T084235/IMG_DATA/T34JFP_20170312T082001_B8A.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170312T082001_N0204_R121_T34JFP_20170312T084235.SAFE/GRANULE/L1C_T34JFP_A008984_20170312T084235/IMG_DATA/T34JFP_20170312T082001_B11.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170312T082001_N0204_R121_T34JFP_20170312T084235.SAFE/GRANULE/L1C_T34JFP_A008984_20170312T084235/IMG_DATA/T34JFP_20170312T082001_B12.jp2'))


date = "2017-03-12"
create_mosaic(fp_bands, sp_bands, date)

first part completed
second part completed
mosaic is created and saved


In [19]:
### 2017-03-22 mosiac

fp_bands = []
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170322T081611_N0204_R121_T34JEP_20170322T084728.SAFE/GRANULE/L1C_T34JEP_A009127_20170322T084728/IMG_DATA/T34JEP_20170322T081611_B02.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170322T081611_N0204_R121_T34JEP_20170322T084728.SAFE/GRANULE/L1C_T34JEP_A009127_20170322T084728/IMG_DATA/T34JEP_20170322T081611_B03.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170322T081611_N0204_R121_T34JEP_20170322T084728.SAFE/GRANULE/L1C_T34JEP_A009127_20170322T084728/IMG_DATA/T34JEP_20170322T081611_B04.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170322T081611_N0204_R121_T34JEP_20170322T084728.SAFE/GRANULE/L1C_T34JEP_A009127_20170322T084728/IMG_DATA/T34JEP_20170322T081611_B05.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170322T081611_N0204_R121_T34JEP_20170322T084728.SAFE/GRANULE/L1C_T34JEP_A009127_20170322T084728/IMG_DATA/T34JEP_20170322T081611_B06.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170322T081611_N0204_R121_T34JEP_20170322T084728.SAFE/GRANULE/L1C_T34JEP_A009127_20170322T084728/IMG_DATA/T34JEP_20170322T081611_B07.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170322T081611_N0204_R121_T34JEP_20170322T084728.SAFE/GRANULE/L1C_T34JEP_A009127_20170322T084728/IMG_DATA/T34JEP_20170322T081611_B08.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170322T081611_N0204_R121_T34JEP_20170322T084728.SAFE/GRANULE/L1C_T34JEP_A009127_20170322T084728/IMG_DATA/T34JEP_20170322T081611_B8A.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170322T081611_N0204_R121_T34JEP_20170322T084728.SAFE/GRANULE/L1C_T34JEP_A009127_20170322T084728/IMG_DATA/T34JEP_20170322T081611_B11.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170322T081611_N0204_R121_T34JEP_20170322T084728.SAFE/GRANULE/L1C_T34JEP_A009127_20170322T084728/IMG_DATA/T34JEP_20170322T081611_B12.jp2'))


sp_bands = []
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170322T081611_N0204_R121_T34JFP_20170322T084728.SAFE/GRANULE/L1C_T34JFP_A009127_20170322T084728/IMG_DATA/T34JFP_20170322T081611_B02.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170322T081611_N0204_R121_T34JFP_20170322T084728.SAFE/GRANULE/L1C_T34JFP_A009127_20170322T084728/IMG_DATA/T34JFP_20170322T081611_B03.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170322T081611_N0204_R121_T34JFP_20170322T084728.SAFE/GRANULE/L1C_T34JFP_A009127_20170322T084728/IMG_DATA/T34JFP_20170322T081611_B04.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170322T081611_N0204_R121_T34JFP_20170322T084728.SAFE/GRANULE/L1C_T34JFP_A009127_20170322T084728/IMG_DATA/T34JFP_20170322T081611_B05.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170322T081611_N0204_R121_T34JFP_20170322T084728.SAFE/GRANULE/L1C_T34JFP_A009127_20170322T084728/IMG_DATA/T34JFP_20170322T081611_B06.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170322T081611_N0204_R121_T34JFP_20170322T084728.SAFE/GRANULE/L1C_T34JFP_A009127_20170322T084728/IMG_DATA/T34JFP_20170322T081611_B07.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170322T081611_N0204_R121_T34JFP_20170322T084728.SAFE/GRANULE/L1C_T34JFP_A009127_20170322T084728/IMG_DATA/T34JFP_20170322T081611_B08.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170322T081611_N0204_R121_T34JFP_20170322T084728.SAFE/GRANULE/L1C_T34JFP_A009127_20170322T084728/IMG_DATA/T34JFP_20170322T081611_B8A.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170322T081611_N0204_R121_T34JFP_20170322T084728.SAFE/GRANULE/L1C_T34JFP_A009127_20170322T084728/IMG_DATA/T34JFP_20170322T081611_B11.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170322T081611_N0204_R121_T34JFP_20170322T084728.SAFE/GRANULE/L1C_T34JFP_A009127_20170322T084728/IMG_DATA/T34JFP_20170322T081611_B12.jp2'))


date = "2017-03-22"
create_mosaic(fp_bands, sp_bands, date)

first part completed
second part completed
mosaic is created and saved


In [ ]:
## 2017-05-31 mosiac

fp_bands = []
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170531T082011_N0205_R121_T34JEP_20170531T084246.SAFE/GRANULE/L1C_T34JEP_A010128_20170531T084246/IMG_DATA/T34JEP_20170531T082011_B02.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170531T082011_N0205_R121_T34JEP_20170531T084246.SAFE/GRANULE/L1C_T34JEP_A010128_20170531T084246/IMG_DATA/T34JEP_20170531T082011_B03.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170531T082011_N0205_R121_T34JEP_20170531T084246.SAFE/GRANULE/L1C_T34JEP_A010128_20170531T084246/IMG_DATA/T34JEP_20170531T082011_B04.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170531T082011_N0205_R121_T34JEP_20170531T084246.SAFE/GRANULE/L1C_T34JEP_A010128_20170531T084246/IMG_DATA/T34JEP_20170531T082011_B05.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170531T082011_N0205_R121_T34JEP_20170531T084246.SAFE/GRANULE/L1C_T34JEP_A010128_20170531T084246/IMG_DATA/T34JEP_20170531T082011_B06.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170531T082011_N0205_R121_T34JEP_20170531T084246.SAFE/GRANULE/L1C_T34JEP_A010128_20170531T084246/IMG_DATA/T34JEP_20170531T082011_B07.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170531T082011_N0205_R121_T34JEP_20170531T084246.SAFE/GRANULE/L1C_T34JEP_A010128_20170531T084246/IMG_DATA/T34JEP_20170531T082011_B08.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170531T082011_N0205_R121_T34JEP_20170531T084246.SAFE/GRANULE/L1C_T34JEP_A010128_20170531T084246/IMG_DATA/T34JEP_20170531T082011_B8A.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170531T082011_N0205_R121_T34JEP_20170531T084246.SAFE/GRANULE/L1C_T34JEP_A010128_20170531T084246/IMG_DATA/T34JEP_20170531T082011_B11.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170531T082011_N0205_R121_T34JEP_20170531T084246.SAFE/GRANULE/L1C_T34JEP_A010128_20170531T084246/IMG_DATA/T34JEP_20170531T082011_B12.jp2'))


sp_bands = []
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170531T082011_N0205_R121_T34JFP_20170531T084246.SAFE/GRANULE/L1C_T34JFP_A010128_20170531T084246/IMG_DATA/T34JFP_20170531T082011_B02.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170531T082011_N0205_R121_T34JFP_20170531T084246.SAFE/GRANULE/L1C_T34JFP_A010128_20170531T084246/IMG_DATA/T34JFP_20170531T082011_B03.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170531T082011_N0205_R121_T34JFP_20170531T084246.SAFE/GRANULE/L1C_T34JFP_A010128_20170531T084246/IMG_DATA/T34JFP_20170531T082011_B04.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170531T082011_N0205_R121_T34JFP_20170531T084246.SAFE/GRANULE/L1C_T34JFP_A010128_20170531T084246/IMG_DATA/T34JFP_20170531T082011_B05.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170531T082011_N0205_R121_T34JFP_20170531T084246.SAFE/GRANULE/L1C_T34JFP_A010128_20170531T084246/IMG_DATA/T34JFP_20170531T082011_B06.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170531T082011_N0205_R121_T34JFP_20170531T084246.SAFE/GRANULE/L1C_T34JFP_A010128_20170531T084246/IMG_DATA/T34JFP_20170531T082011_B07.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170531T082011_N0205_R121_T34JFP_20170531T084246.SAFE/GRANULE/L1C_T34JFP_A010128_20170531T084246/IMG_DATA/T34JFP_20170531T082011_B08.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170531T082011_N0205_R121_T34JFP_20170531T084246.SAFE/GRANULE/L1C_T34JFP_A010128_20170531T084246/IMG_DATA/T34JFP_20170531T082011_B8A.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170531T082011_N0205_R121_T34JFP_20170531T084246.SAFE/GRANULE/L1C_T34JFP_A010128_20170531T084246/IMG_DATA/T34JFP_20170531T082011_B11.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170531T082011_N0205_R121_T34JFP_20170531T084246.SAFE/GRANULE/L1C_T34JFP_A010128_20170531T084246/IMG_DATA/T34JFP_20170531T082011_B12.jp2'))

date = "2017-05-31"
create_mosaic(fp_bands, sp_bands, date)

In [ ]:
## 2017-06-20 mosiac

fp_bands = []
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170620T082011_N0205_R121_T34JEP_20170620T084200.SAFE/GRANULE/L1C_T34JEP_A010414_20170620T084200/IMG_DATA/T34JEP_20170620T082011_B02.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170620T082011_N0205_R121_T34JEP_20170620T084200.SAFE/GRANULE/L1C_T34JEP_A010414_20170620T084200/IMG_DATA/T34JEP_20170620T082011_B03.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170620T082011_N0205_R121_T34JEP_20170620T084200.SAFE/GRANULE/L1C_T34JEP_A010414_20170620T084200/IMG_DATA/T34JEP_20170620T082011_B04.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170620T082011_N0205_R121_T34JEP_20170620T084200.SAFE/GRANULE/L1C_T34JEP_A010414_20170620T084200/IMG_DATA/T34JEP_20170620T082011_B05.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170620T082011_N0205_R121_T34JEP_20170620T084200.SAFE/GRANULE/L1C_T34JEP_A010414_20170620T084200/IMG_DATA/T34JEP_20170620T082011_B06.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170620T082011_N0205_R121_T34JEP_20170620T084200.SAFE/GRANULE/L1C_T34JEP_A010414_20170620T084200/IMG_DATA/T34JEP_20170620T082011_B07.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170620T082011_N0205_R121_T34JEP_20170620T084200.SAFE/GRANULE/L1C_T34JEP_A010414_20170620T084200/IMG_DATA/T34JEP_20170620T082011_B08.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170620T082011_N0205_R121_T34JEP_20170620T084200.SAFE/GRANULE/L1C_T34JEP_A010414_20170620T084200/IMG_DATA/T34JEP_20170620T082011_B8A.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170620T082011_N0205_R121_T34JEP_20170620T084200.SAFE/GRANULE/L1C_T34JEP_A010414_20170620T084200/IMG_DATA/T34JEP_20170620T082011_B11.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170620T082011_N0205_R121_T34JEP_20170620T084200.SAFE/GRANULE/L1C_T34JEP_A010414_20170620T084200/IMG_DATA/T34JEP_20170620T082011_B12.jp2'))

sp_bands = []
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170620T082011_N0205_R121_T34JFP_20170620T084200.SAFE/GRANULE/L1C_T34JFP_A010414_20170620T084200/IMG_DATA/T34JFP_20170620T082011_B02.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170620T082011_N0205_R121_T34JFP_20170620T084200.SAFE/GRANULE/L1C_T34JFP_A010414_20170620T084200/IMG_DATA/T34JFP_20170620T082011_B03.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170620T082011_N0205_R121_T34JFP_20170620T084200.SAFE/GRANULE/L1C_T34JFP_A010414_20170620T084200/IMG_DATA/T34JFP_20170620T082011_B04.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170620T082011_N0205_R121_T34JFP_20170620T084200.SAFE/GRANULE/L1C_T34JFP_A010414_20170620T084200/IMG_DATA/T34JFP_20170620T082011_B05.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170620T082011_N0205_R121_T34JFP_20170620T084200.SAFE/GRANULE/L1C_T34JFP_A010414_20170620T084200/IMG_DATA/T34JFP_20170620T082011_B06.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170620T082011_N0205_R121_T34JFP_20170620T084200.SAFE/GRANULE/L1C_T34JFP_A010414_20170620T084200/IMG_DATA/T34JFP_20170620T082011_B07.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170620T082011_N0205_R121_T34JFP_20170620T084200.SAFE/GRANULE/L1C_T34JFP_A010414_20170620T084200/IMG_DATA/T34JFP_20170620T082011_B08.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170620T082011_N0205_R121_T34JFP_20170620T084200.SAFE/GRANULE/L1C_T34JFP_A010414_20170620T084200/IMG_DATA/T34JFP_20170620T082011_B8A.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170620T082011_N0205_R121_T34JFP_20170620T084200.SAFE/GRANULE/L1C_T34JFP_A010414_20170620T084200/IMG_DATA/T34JFP_20170620T082011_B11.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170620T082011_N0205_R121_T34JFP_20170620T084200.SAFE/GRANULE/L1C_T34JFP_A010414_20170620T084200/IMG_DATA/T34JFP_20170620T082011_B12.jp2'))

date = "2017-06-20"
create_mosaic(fp_bands, sp_bands, date)

In [ ]:
## 2017-07-10 mosiac

fp_bands = []
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170710T082011_N0205_R121_T34JEP_20170710T084244.SAFE/GRANULE/L1C_T34JEP_A010700_20170710T084244/IMG_DATA/T34JEP_20170710T082011_B02.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170710T082011_N0205_R121_T34JEP_20170710T084244.SAFE/GRANULE/L1C_T34JEP_A010700_20170710T084244/IMG_DATA/T34JEP_20170710T082011_B03.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170710T082011_N0205_R121_T34JEP_20170710T084244.SAFE/GRANULE/L1C_T34JEP_A010700_20170710T084244/IMG_DATA/T34JEP_20170710T082011_B04.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170710T082011_N0205_R121_T34JEP_20170710T084244.SAFE/GRANULE/L1C_T34JEP_A010700_20170710T084244/IMG_DATA/T34JEP_20170710T082011_B05.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170710T082011_N0205_R121_T34JEP_20170710T084244.SAFE/GRANULE/L1C_T34JEP_A010700_20170710T084244/IMG_DATA/T34JEP_20170710T082011_B06.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170710T082011_N0205_R121_T34JEP_20170710T084244.SAFE/GRANULE/L1C_T34JEP_A010700_20170710T084244/IMG_DATA/T34JEP_20170710T082011_B07.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170710T082011_N0205_R121_T34JEP_20170710T084244.SAFE/GRANULE/L1C_T34JEP_A010700_20170710T084244/IMG_DATA/T34JEP_20170710T082011_B08.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170710T082011_N0205_R121_T34JEP_20170710T084244.SAFE/GRANULE/L1C_T34JEP_A010700_20170710T084244/IMG_DATA/T34JEP_20170710T082011_B8A.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170710T082011_N0205_R121_T34JEP_20170710T084244.SAFE/GRANULE/L1C_T34JEP_A010700_20170710T084244/IMG_DATA/T34JEP_20170710T082011_B11.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170710T082011_N0205_R121_T34JEP_20170710T084244.SAFE/GRANULE/L1C_T34JEP_A010700_20170710T084244/IMG_DATA/T34JEP_20170710T082011_B12.jp2'))


sp_bands = []
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170710T082011_N0205_R121_T34JFP_20170710T084244.SAFE/GRANULE/L1C_T34JFP_A010700_20170710T084244/IMG_DATA/T34JFP_20170710T082011_B02.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170710T082011_N0205_R121_T34JFP_20170710T084244.SAFE/GRANULE/L1C_T34JFP_A010700_20170710T084244/IMG_DATA/T34JFP_20170710T082011_B03.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170710T082011_N0205_R121_T34JFP_20170710T084244.SAFE/GRANULE/L1C_T34JFP_A010700_20170710T084244/IMG_DATA/T34JFP_20170710T082011_B04.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170710T082011_N0205_R121_T34JFP_20170710T084244.SAFE/GRANULE/L1C_T34JFP_A010700_20170710T084244/IMG_DATA/T34JFP_20170710T082011_B05.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170710T082011_N0205_R121_T34JFP_20170710T084244.SAFE/GRANULE/L1C_T34JFP_A010700_20170710T084244/IMG_DATA/T34JFP_20170710T082011_B06.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170710T082011_N0205_R121_T34JFP_20170710T084244.SAFE/GRANULE/L1C_T34JFP_A010700_20170710T084244/IMG_DATA/T34JFP_20170710T082011_B07.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170710T082011_N0205_R121_T34JFP_20170710T084244.SAFE/GRANULE/L1C_T34JFP_A010700_20170710T084244/IMG_DATA/T34JFP_20170710T082011_B08.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170710T082011_N0205_R121_T34JFP_20170710T084244.SAFE/GRANULE/L1C_T34JFP_A010700_20170710T084244/IMG_DATA/T34JFP_20170710T082011_B8A.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170710T082011_N0205_R121_T34JFP_20170710T084244.SAFE/GRANULE/L1C_T34JFP_A010700_20170710T084244/IMG_DATA/T34JFP_20170710T082011_B11.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170710T082011_N0205_R121_T34JFP_20170710T084244.SAFE/GRANULE/L1C_T34JFP_A010700_20170710T084244/IMG_DATA/T34JFP_20170710T082011_B12.jp2'))


date = "2017-07-10"
create_mosaic(fp_bands, sp_bands, date)

In [ ]:
## 2017-07-15 mosiac

fp_bands = []
fp_bands.append(os.path.join(parent_path, 'S2B_MSIL1C_20170715T081609_N0205_R121_T34JEP_20170715T084650.SAFE/GRANULE/L1C_T34JEP_A001863_20170715T084650/IMG_DATA/T34JEP_20170715T081609_B02.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2B_MSIL1C_20170715T081609_N0205_R121_T34JEP_20170715T084650.SAFE/GRANULE/L1C_T34JEP_A001863_20170715T084650/IMG_DATA/T34JEP_20170715T081609_B03.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2B_MSIL1C_20170715T081609_N0205_R121_T34JEP_20170715T084650.SAFE/GRANULE/L1C_T34JEP_A001863_20170715T084650/IMG_DATA/T34JEP_20170715T081609_B04.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2B_MSIL1C_20170715T081609_N0205_R121_T34JEP_20170715T084650.SAFE/GRANULE/L1C_T34JEP_A001863_20170715T084650/IMG_DATA/T34JEP_20170715T081609_B05.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2B_MSIL1C_20170715T081609_N0205_R121_T34JEP_20170715T084650.SAFE/GRANULE/L1C_T34JEP_A001863_20170715T084650/IMG_DATA/T34JEP_20170715T081609_B06.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2B_MSIL1C_20170715T081609_N0205_R121_T34JEP_20170715T084650.SAFE/GRANULE/L1C_T34JEP_A001863_20170715T084650/IMG_DATA/T34JEP_20170715T081609_B07.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2B_MSIL1C_20170715T081609_N0205_R121_T34JEP_20170715T084650.SAFE/GRANULE/L1C_T34JEP_A001863_20170715T084650/IMG_DATA/T34JEP_20170715T081609_B08.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2B_MSIL1C_20170715T081609_N0205_R121_T34JEP_20170715T084650.SAFE/GRANULE/L1C_T34JEP_A001863_20170715T084650/IMG_DATA/T34JEP_20170715T081609_B8A.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2B_MSIL1C_20170715T081609_N0205_R121_T34JEP_20170715T084650.SAFE/GRANULE/L1C_T34JEP_A001863_20170715T084650/IMG_DATA/T34JEP_20170715T081609_B11.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2B_MSIL1C_20170715T081609_N0205_R121_T34JEP_20170715T084650.SAFE/GRANULE/L1C_T34JEP_A001863_20170715T084650/IMG_DATA/T34JEP_20170715T081609_B12.jp2'))


sp_bands = []
sp_bands.append(os.path.join(parent_path, 'S2B_MSIL1C_20170715T081609_N0205_R121_T34JFP_20170715T084650.SAFE/GRANULE/L1C_T34JFP_A001863_20170715T084650/IMG_DATA/T34JFP_20170715T081609_B02.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2B_MSIL1C_20170715T081609_N0205_R121_T34JFP_20170715T084650.SAFE/GRANULE/L1C_T34JFP_A001863_20170715T084650/IMG_DATA/T34JFP_20170715T081609_B03.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2B_MSIL1C_20170715T081609_N0205_R121_T34JFP_20170715T084650.SAFE/GRANULE/L1C_T34JFP_A001863_20170715T084650/IMG_DATA/T34JFP_20170715T081609_B04.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2B_MSIL1C_20170715T081609_N0205_R121_T34JFP_20170715T084650.SAFE/GRANULE/L1C_T34JFP_A001863_20170715T084650/IMG_DATA/T34JFP_20170715T081609_B05.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2B_MSIL1C_20170715T081609_N0205_R121_T34JFP_20170715T084650.SAFE/GRANULE/L1C_T34JFP_A001863_20170715T084650/IMG_DATA/T34JFP_20170715T081609_B06.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2B_MSIL1C_20170715T081609_N0205_R121_T34JFP_20170715T084650.SAFE/GRANULE/L1C_T34JFP_A001863_20170715T084650/IMG_DATA/T34JFP_20170715T081609_B07.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2B_MSIL1C_20170715T081609_N0205_R121_T34JFP_20170715T084650.SAFE/GRANULE/L1C_T34JFP_A001863_20170715T084650/IMG_DATA/T34JFP_20170715T081609_B08.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2B_MSIL1C_20170715T081609_N0205_R121_T34JFP_20170715T084650.SAFE/GRANULE/L1C_T34JFP_A001863_20170715T084650/IMG_DATA/T34JFP_20170715T081609_B8A.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2B_MSIL1C_20170715T081609_N0205_R121_T34JFP_20170715T084650.SAFE/GRANULE/L1C_T34JFP_A001863_20170715T084650/IMG_DATA/T34JFP_20170715T081609_B11.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2B_MSIL1C_20170715T081609_N0205_R121_T34JFP_20170715T084650.SAFE/GRANULE/L1C_T34JFP_A001863_20170715T084650/IMG_DATA/T34JFP_20170715T081609_B12.jp2'))


date = "2017-07-15"
create_mosaic(fp_bands, sp_bands, date)

In [ ]:
## 2017-08-04 mosiac

fp_bands=[]
fp_bands.append(os.path.join(parent_path, 'S2B_MSIL1C_20170804T081559_N0205_R121_T34JEP_20170804T084631.SAFE/GRANULE/L1C_T34JEP_A002149_20170804T084631/IMG_DATA/T34JEP_20170804T081559_B02.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2B_MSIL1C_20170804T081559_N0205_R121_T34JEP_20170804T084631.SAFE/GRANULE/L1C_T34JEP_A002149_20170804T084631/IMG_DATA/T34JEP_20170804T081559_B03.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2B_MSIL1C_20170804T081559_N0205_R121_T34JEP_20170804T084631.SAFE/GRANULE/L1C_T34JEP_A002149_20170804T084631/IMG_DATA/T34JEP_20170804T081559_B04.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2B_MSIL1C_20170804T081559_N0205_R121_T34JEP_20170804T084631.SAFE/GRANULE/L1C_T34JEP_A002149_20170804T084631/IMG_DATA/T34JEP_20170804T081559_B05.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2B_MSIL1C_20170804T081559_N0205_R121_T34JEP_20170804T084631.SAFE/GRANULE/L1C_T34JEP_A002149_20170804T084631/IMG_DATA/T34JEP_20170804T081559_B06.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2B_MSIL1C_20170804T081559_N0205_R121_T34JEP_20170804T084631.SAFE/GRANULE/L1C_T34JEP_A002149_20170804T084631/IMG_DATA/T34JEP_20170804T081559_B07.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2B_MSIL1C_20170804T081559_N0205_R121_T34JEP_20170804T084631.SAFE/GRANULE/L1C_T34JEP_A002149_20170804T084631/IMG_DATA/T34JEP_20170804T081559_B08.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2B_MSIL1C_20170804T081559_N0205_R121_T34JEP_20170804T084631.SAFE/GRANULE/L1C_T34JEP_A002149_20170804T084631/IMG_DATA/T34JEP_20170804T081559_B8A.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2B_MSIL1C_20170804T081559_N0205_R121_T34JEP_20170804T084631.SAFE/GRANULE/L1C_T34JEP_A002149_20170804T084631/IMG_DATA/T34JEP_20170804T081559_B11.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2B_MSIL1C_20170804T081559_N0205_R121_T34JEP_20170804T084631.SAFE/GRANULE/L1C_T34JEP_A002149_20170804T084631/IMG_DATA/T34JEP_20170804T081559_B12.jp2'))


sp_bands = []
sp_bands.append(os.path.join(parent_path, 'S2B_MSIL1C_20170804T081559_N0205_R121_T34JFP_20170804T084631.SAFE/GRANULE/L1C_T34JFP_A002149_20170804T084631/IMG_DATA/T34JFP_20170804T081559_B02.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2B_MSIL1C_20170804T081559_N0205_R121_T34JFP_20170804T084631.SAFE/GRANULE/L1C_T34JFP_A002149_20170804T084631/IMG_DATA/T34JFP_20170804T081559_B03.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2B_MSIL1C_20170804T081559_N0205_R121_T34JFP_20170804T084631.SAFE/GRANULE/L1C_T34JFP_A002149_20170804T084631/IMG_DATA/T34JFP_20170804T081559_B04.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2B_MSIL1C_20170804T081559_N0205_R121_T34JFP_20170804T084631.SAFE/GRANULE/L1C_T34JFP_A002149_20170804T084631/IMG_DATA/T34JFP_20170804T081559_B05.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2B_MSIL1C_20170804T081559_N0205_R121_T34JFP_20170804T084631.SAFE/GRANULE/L1C_T34JFP_A002149_20170804T084631/IMG_DATA/T34JFP_20170804T081559_B06.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2B_MSIL1C_20170804T081559_N0205_R121_T34JFP_20170804T084631.SAFE/GRANULE/L1C_T34JFP_A002149_20170804T084631/IMG_DATA/T34JFP_20170804T081559_B07.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2B_MSIL1C_20170804T081559_N0205_R121_T34JFP_20170804T084631.SAFE/GRANULE/L1C_T34JFP_A002149_20170804T084631/IMG_DATA/T34JFP_20170804T081559_B08.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2B_MSIL1C_20170804T081559_N0205_R121_T34JFP_20170804T084631.SAFE/GRANULE/L1C_T34JFP_A002149_20170804T084631/IMG_DATA/T34JFP_20170804T081559_B8A.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2B_MSIL1C_20170804T081559_N0205_R121_T34JFP_20170804T084631.SAFE/GRANULE/L1C_T34JFP_A002149_20170804T084631/IMG_DATA/T34JFP_20170804T081559_B11.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2B_MSIL1C_20170804T081559_N0205_R121_T34JFP_20170804T084631.SAFE/GRANULE/L1C_T34JFP_A002149_20170804T084631/IMG_DATA/T34JFP_20170804T081559_B12.jp2'))

date = "2017-08-04"
create_mosaic(fp_bands, sp_bands, date)

In [ ]:
## 2017-08-19 mosiac

fp_bands = []
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170819T082011_N0205_R121_T34JEP_20170819T084427.SAFE/GRANULE/L1C_T34JEP_A011272_20170819T084427/IMG_DATA/T34JEP_20170819T082011_B02.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170819T082011_N0205_R121_T34JEP_20170819T084427.SAFE/GRANULE/L1C_T34JEP_A011272_20170819T084427/IMG_DATA/T34JEP_20170819T082011_B03.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170819T082011_N0205_R121_T34JEP_20170819T084427.SAFE/GRANULE/L1C_T34JEP_A011272_20170819T084427/IMG_DATA/T34JEP_20170819T082011_B04.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170819T082011_N0205_R121_T34JEP_20170819T084427.SAFE/GRANULE/L1C_T34JEP_A011272_20170819T084427/IMG_DATA/T34JEP_20170819T082011_B05.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170819T082011_N0205_R121_T34JEP_20170819T084427.SAFE/GRANULE/L1C_T34JEP_A011272_20170819T084427/IMG_DATA/T34JEP_20170819T082011_B06.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170819T082011_N0205_R121_T34JEP_20170819T084427.SAFE/GRANULE/L1C_T34JEP_A011272_20170819T084427/IMG_DATA/T34JEP_20170819T082011_B07.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170819T082011_N0205_R121_T34JEP_20170819T084427.SAFE/GRANULE/L1C_T34JEP_A011272_20170819T084427/IMG_DATA/T34JEP_20170819T082011_B08.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170819T082011_N0205_R121_T34JEP_20170819T084427.SAFE/GRANULE/L1C_T34JEP_A011272_20170819T084427/IMG_DATA/T34JEP_20170819T082011_B8A.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170819T082011_N0205_R121_T34JEP_20170819T084427.SAFE/GRANULE/L1C_T34JEP_A011272_20170819T084427/IMG_DATA/T34JEP_20170819T082011_B11.jp2'))
fp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170819T082011_N0205_R121_T34JEP_20170819T084427.SAFE/GRANULE/L1C_T34JEP_A011272_20170819T084427/IMG_DATA/T34JEP_20170819T082011_B12.jp2'))

sp_bands = []
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170819T082011_N0205_R121_T34JFP_20170819T084427.SAFE/GRANULE/L1C_T34JFP_A011272_20170819T084427/IMG_DATA/T34JFP_20170819T082011_B02.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170819T082011_N0205_R121_T34JFP_20170819T084427.SAFE/GRANULE/L1C_T34JFP_A011272_20170819T084427/IMG_DATA/T34JFP_20170819T082011_B03.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170819T082011_N0205_R121_T34JFP_20170819T084427.SAFE/GRANULE/L1C_T34JFP_A011272_20170819T084427/IMG_DATA/T34JFP_20170819T082011_B04.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170819T082011_N0205_R121_T34JFP_20170819T084427.SAFE/GRANULE/L1C_T34JFP_A011272_20170819T084427/IMG_DATA/T34JFP_20170819T082011_B05.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170819T082011_N0205_R121_T34JFP_20170819T084427.SAFE/GRANULE/L1C_T34JFP_A011272_20170819T084427/IMG_DATA/T34JFP_20170819T082011_B06.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170819T082011_N0205_R121_T34JFP_20170819T084427.SAFE/GRANULE/L1C_T34JFP_A011272_20170819T084427/IMG_DATA/T34JFP_20170819T082011_B07.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170819T082011_N0205_R121_T34JFP_20170819T084427.SAFE/GRANULE/L1C_T34JFP_A011272_20170819T084427/IMG_DATA/T34JFP_20170819T082011_B08.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170819T082011_N0205_R121_T34JFP_20170819T084427.SAFE/GRANULE/L1C_T34JFP_A011272_20170819T084427/IMG_DATA/T34JFP_20170819T082011_B8A.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170819T082011_N0205_R121_T34JFP_20170819T084427.SAFE/GRANULE/L1C_T34JFP_A011272_20170819T084427/IMG_DATA/T34JFP_20170819T082011_B11.jp2'))
sp_bands.append(os.path.join(parent_path, 'S2A_MSIL1C_20170819T082011_N0205_R121_T34JFP_20170819T084427.SAFE/GRANULE/L1C_T34JFP_A011272_20170819T084427/IMG_DATA/T34JFP_20170819T082011_B12.jp2'))


date = "2017-08-19"
create_mosaic(fp_bands, sp_bands, date)

In [ ]:
train_fe['area'] = train_fe['geometry'].area
train_fe['centroid'] = train_fe['geometry'].centroid
test_fe['area'] = test_fe['geometry'].area
test_fe['centroid'] = test_fe['geometry'].centroid